# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

In [1]:
# your code here
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
from urllib.request import urlopen

## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

In [6]:
# your code here
url = 'https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset=0'
response = urlopen(url)
results = json.load(response)

#flattened_data = json_normalize(results)
#flattened_data1 = json_normalize(flattened_data.articles[0])
#flattened_data

#total page
total_pages = results['pagination']['page_count']

df=pd.DataFrame()
for i in range(total_pages):
    k=84*i
    url=f'https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset={k}&sort=sale'
    response = urlopen(url)
    results = json.load(response)
    flattened_data = json_normalize(results)
    flattened_data1 = json_normalize(flattened_data.articles[0])
    #flattened_data1 = flattened_data1.set_index('sku')
    df = df.append(flattened_data1)
    
    #if i == 10:
    #    break

df

<ipython-input-6-8c064cb8ef64>:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  flattened_data = json_normalize(results)
<ipython-input-6-8c064cb8ef64>:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  flattened_data1 = json_normalize(flattened_data.articles[0])


,sku,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,amount,condition,condition_key
0,NI114D0CS-A11,COURT BOROUGH - Baskets basses - white,"[27.5, 28, 28.5, 29.5, 30, 31, 31.5, 32, 33, 3...",nike-sportswear-court-borough-baskets-basses-n...,[{'path': 'spp-media-p1/0b5fec9e420b37099ccfe6...,Nike Sportswear,False,[],"[{'key': 'discountRate', 'value': 'Jusqu’à -25...",shoe,[],"34,95 €","26,15 €",True,False,True,False,NaN,NaN,NaN
1,M9123L0BG-A11,CACHITO8 - Veste d'hiver - cremeweiß,"[6a, 8a, 9a, 10a, 11-12a]",mango-cachito8-veste-dhiver-cremeweiss-m9123l0...,[{'path': 'spp-media-p1/35236b50637935189937f5...,Mango,False,[],"[{'key': 'discountRate', 'value': '-67%', 'tra...",clothing,[],"59,99 €","19,99 €",False,False,False,False,NaN,NaN,NaN
2,LE226G005-A11,BATWING TEE - T-shirt imprimé - white,"[2a, 3a, 4a, 5a, 8a, 10a, 12a, 14a, 16a]",levisr-t-shirt-imprime-white-le226g005-a11,[{'path': 'spp-media-p1/8d33b25d4a1b3a718e588e...,Levi's®,False,[],"[{'key': 'discountRate', 'value': '-15%', 'tra...",clothing,[],"17,95 €","15,25 €",True,False,True,False,NaN,NaN,NaN
3,TO184A02M-K11,2 PACK - T-shirt basique - blue,"[8-10a, 10-12a, 12-14a, 14-16a]",tommy-hilfiger-tee-2-pack-caraco-blue-to184a02...,[{'path': 'spp-media-p1/306c5fc7a0b339988ca6db...,Tommy Hilfiger,False,[],"[{'key': 'discountRate', 'value': '-5%', 'trac...",clothing,[],"27,95 €","26,65 €",False,False,False,False,NaN,NaN,NaN
4,JAN24B004-Q11,Pantalon cargo - black,"[8a, 9a, 10a, 11a, 12a, 13a, 14a, 15a, 16a]",jackandjones-junior-pantalon-cargo-black-jan24...,[{'path': 'spp-media-p1/6272154bdaec3231bf44b9...,Jack & Jones Junior,False,[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",clothing,[],"49,95 €","34,95 €",False,False,False,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,JH186A01J-B11,SHORT SLEEVES - Body - sesame melange,[0-1m],joha-short-sleeves-body-sesame-melange-jh186a0...,[{'path': 'spp-media-p1/24dfa01a3a4c3953bb0713...,Joha,False,[],"[{'key': 'campaign', 'value': 'Prix Mini', 'tr...",underwear,[],"16,95 €","16,95 €",False,False,False,False,NaN,NaN,NaN
47,JH186A01A-K11,BABY - Body - blue,[0-1m],joha-baby-body-jh186a01a-k11,[{'path': 'spp-media-p1/38f70b384e1a3b849da4bd...,Joha,False,[],"[{'key': 'campaign', 'value': 'Prix Mini', 'tr...",underwear,"[{'key': 'slow_delivery_flag', 'label': 'Livra...","12,95 €","12,95 €",False,False,False,False,NaN,NaN,NaN
48,VA213D00X-J11,SK8 MTE - Baskets montantes - lamingo pink/hol...,[32],vans-sk8-mte-baskets-montantes-lamingo-pinkhol...,[{'path': 'spp-media-p1/41155f0ebe9d3881b9fb06...,Vans,False,[],"[{'key': 'campaign', 'value': 'Prix Mini', 'tr...",shoe,"[{'key': 'slow_delivery_flag', 'label': 'Livra...","69,95 €","69,95 €",False,False,False,False,NaN,NaN,NaN
49,LU213D02D-B11,YARA TEX - Baskets montantes - stone,[30],lurchi-yara-tex-baskets-montantes-stone-lu213d...,[{'path': 'spp-media-p1/dc672bb39002307ba57bee...,Lurchi,False,[],"[{'key': 'campaign', 'value': 'Prix Mini', 'tr...",shoe,"[{'key': 'slow_delivery_flag', 'label': 'Livra...","74,95 €","74,95 €",False,False,False,False,NaN,NaN,NaN


## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

In [ ]:
# your code here